In [1]:
from glob import glob
import os
import torch
import torchmetrics
import nibabel as nib
import subprocess
import numpy as np
import torchmetrics
import pandas as pd

In [2]:
region_names_map6 = ["brain", "hy", "mb", "hpf", "str", "ctx", "gp", "th", "cb", "olf", "bs"]
region_names_svbp = ["brain", "olf", "ctx", "hpf", "str", "gp", "hy", "th", "mb", "cb", "bs"]

In [3]:
atlas_mask = "dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz"
atlas_labels = "dataset3/Atlas/atlas_gin_map6.nii.gz"

raw_mris = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*_id.nii.gz')))
raw_masks = sorted(glob(os.path.join('dataset3', 'GIN', 'Mask', '*_id.nii.gz')))
raw_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', '*_id.nii.gz')))

affine_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_affine_convert.nii.gz")))
deep1_labels = sorted(glob(os.path.join('output', 'GIN', 'scenario1', 'Labels', '*.nii.gz')))
deep2_labels = sorted(glob(os.path.join('output', 'GIN', 'scenario2', 'Labels', '*.nii.gz')))
deep3_labels = sorted(glob(os.path.join('output', 'GIN', 'scenario3', 'Labels', '*.nii.gz')))
ants_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_deformable_convert.nii.gz")))
mean_labels = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'Labels', "*_deformable_convert.nii.gz")))

affine_fwd_warps = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
deep1_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario1', 'DeformableWarp', "*_warp.nii.gz")))
deep2_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario2', 'DeformableWarp', "*_warp.nii.gz")))
deep3_fwd_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario3', 'DeformableWarp', "*_warp.nii.gz")))
ants_fwd_warps = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', "*_warp.nii.gz")))
mean_fwd_warps = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', "*_warp.nii.gz")))

deep1_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario1', 'DeformableWarp', "*_invwarp.nii.gz")))
deep2_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario2', 'DeformableWarp', "*_invwarp.nii.gz")))
deep3_bck_warps = sorted(glob(os.path.join('output', 'GIN', 'scenario3', 'DeformableWarp', "*_invwarp.nii.gz")))
ants_bck_warps = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', "*_warp_inverse.nii.gz")))
mean_bck_warps = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', "*_warp_inverse.nii.gz")))

dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=11)

mapping_csv = "dataset3/GIN/labels_mapping.csv"
data = pd.read_csv(mapping_csv)
    
count = 0

list_dict = []
names = ["Raw", "Affine", "ANTS", "OP", "DL", "DL+OP", "ANTS-Mean"]

for i, _ in enumerate(raw_mris):  
    count += 1
    values_raw = [0,0,0,0,0,0,0,0,0,0,0]
    values_affine = [0,0,0,0,0,0,0,0,0,0,0]
    values_ants = [0,0,0,0,0,0,0,0,0,0,0]
    values_deep1 = [0,0,0,0,0,0,0,0,0,0,0]
    values_deep2 = [0,0,0,0,0,0,0,0,0,0,0]
    values_deep3 = [0,0,0,0,0,0,0,0,0,0,0]
    values_mean = [0,0,0,0,0,0,0,0,0,0,0]

    masks = [raw_masks[i]]
    labels = [raw_labels[i]]
    output = 'mri_mask_affine.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_mask])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'NearestNeighbor'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    subprocess.call(command) 
    masks.append(output)
    output = 'mri_labels_affine.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_labels])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    subprocess.call(command) 
    labels.append(output)    
    
    output = 'mri_mask_ants.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_mask])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'NearestNeighbor'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', ants_bck_warps[i]])
    subprocess.call(command) 
    masks.append(output)
    output = 'mri_labels_ants.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_labels])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', ants_bck_warps[i]])
    subprocess.call(command) 
    labels.append(output)
    
    output = 'mri_mask_deep1.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_mask])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'NearestNeighbor'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', deep1_bck_warps[i]])
    subprocess.call(command)     
    masks.append(output)
    output = 'mri_labels_deep1.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_labels])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', deep1_bck_warps[i]])
    subprocess.call(command) 
    labels.append(output)
    
    output = 'mri_mask_deep2.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_mask])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'NearestNeighbor'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', deep2_bck_warps[i]])
    subprocess.call(command)   
    masks.append(output)
    output = 'mri_labels_deep2.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_labels])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', deep2_bck_warps[i]])
    subprocess.call(command) 
    labels.append(output)
    
    output = 'mri_mask_deep3.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_mask])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'NearestNeighbor'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', deep3_bck_warps[i]])
    subprocess.call(command)
    masks.append(output)    
    output = 'mri_labels_deep3.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_labels])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', deep3_bck_warps[i]])
    subprocess.call(command) 
    labels.append(output)
    
    output = 'mri_mask_mean.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_mask])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'NearestNeighbor'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command) 
    masks.append(output)
    output = 'mri_labels_mean.nii.gz'
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_labels])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_fwd_warps[i] + ',1]'])
    command.extend(['-t', mean_bck_warps[i]])
    subprocess.call(command)
    labels.append(output) 
    
    values_list = [values_raw, values_affine, values_ants, values_deep1, values_deep2, values_deep3, values_mean]
    for j in range(len(values_list)):
        mask = nib.load(masks[j]).get_fdata()   
        label = nib.load(labels[j]).get_fdata()  
        values_list[j][0] = (mask==1).sum()*0.12**3 
        for k in range(1,11):
            values_list[j][k] = (label==k).sum()*0.12**3
        
    name = ants_labels[i].split('/')[-1].split('_')[0]    
    datatype = "MAP6" if "M" in name else "SVBP"
    if datatype == "MAP6":
        region_names = region_names_map6
    else:
        region_names = region_names_svbp
    raw_dict = {"raw_volume_" + str(region_names[k]): values_raw[k] for k in range(len(region_names))}
    affine_dict = {"affine_volume_" + str(region_names_map6[k]): values_affine[k] for k in range(len(region_names_map6))}
    ants_dict = {"ants_volume_" + str(region_names_map6[k]): values_ants[k] for k in range(len(region_names_map6))}
    deep1_dict = {"deep1_volume_" + str(region_names_map6[k]): values_deep1[k] for k in range(len(region_names_map6))}
    deep2_dict = {"deep2_volume_" + str(region_names_map6[k]): values_deep2[k] for k in range(len(region_names_map6))}
    deep3_dict = {"deep3_volume_" + str(region_names_map6[k]): values_deep3[k] for k in range(len(region_names_map6))}
    mean_dict = {"antsmean_volume_" + str(region_names_map6[k]): values_mean[k] for k in range(len(region_names_map6))}
    ith_dict = {"name": name, "type": datatype}
    ith_dict.update(raw_dict)
    ith_dict.update(affine_dict)
    ith_dict.update(ants_dict)
    ith_dict.update(deep1_dict)
    ith_dict.update(deep2_dict)
    ith_dict.update(deep3_dict)
    ith_dict.update(mean_dict)
    list_dict.append(ith_dict)
#print(ith_dict)
df = pd.DataFrame(list_dict)
df.to_csv("volume_results.csv")


In [4]:
print(df)

     name  type  raw_volume_brain  raw_volume_olf  raw_volume_ctx  \
0     653  SVBP        486.974592       23.863680      145.093248   
1     655  SVBP        459.095040       20.875968      125.394048   
2     656  SVBP        442.072512       21.225024      130.310208   
3     660  SVBP        451.797696       20.818944      138.487104   
4     661  SVBP        440.569152       19.256832      133.847424   
5     662  SVBP        462.435264       21.446208      139.214592   
6     663  SVBP        467.182080       22.287744      141.283008   
7     664  SVBP        464.455296       23.798016      137.545344   
8     665  SVBP        430.330752       20.782656      133.254720   
9     667  SVBP        422.680896       19.892736      132.266304   
10    668  SVBP        440.636544       19.362240      138.457728   
11    670  SVBP        437.681664       22.204800      135.297216   
12    675  SVBP        423.099072       21.126528      122.990400   
13    678  SVBP        471.014784 